In [12]:
import numpy as np
import math
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw, ImageChops
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
import tensorflow as tf
from tensorflow.python.framework import ops

cellsize = 10


In [34]:
# compute glyph difference in TF
def glyphDiffTF(y_true, y_pred):
    #print(y_true, y_pred)
    #take Y values (1D list of angles, distances) representing outline of glyph and turn into 2D bool image 
    scale = cellsize * 0.8
    angles = y_true[::2]
    dists = y_true[1::2]
    
    xs = tf.cumsum(tf.abs(dists)*tf.cos(angles*math.pi)*scale)
    x1s = xs[:-1]
    x2s = xs[1:]
    ys = tf.cumsum(tf.abs(dists)*tf.sin(angles*math.pi)*scale)
    y1s = ys[:-1]
    y2s = ys[1:]

    xx1s=tf.expand_dims(x1s,-1)
    xx2s=tf.expand_dims(x2s,-1)
    yy1s=tf.expand_dims(y1s,-1)
    yy2s=tf.expand_dims(y2s,-1)

    #build list of x for y in range (0,cellsize)
    ygrid, _ = tf.meshgrid(tf.range(0, cellsize), [0])
    yys = tf.to_float(ygrid)
    xxs = (yys-yy1s)/(yy2s-yy1s)*(xx2s-xx1s)+xx1s
    
    print('xxs',xxs)
    #xxs = tf.map_fn(
    #        #lambda x1, x2, y1, y2, yy: (yy-y1)/(y2-y1)*(x2-x1) if y1<yy and yy<y2 or y2<yy and yy<y1 else 0,
    #        lambda xxyy: (xxyy[4]-xxyy[2])/(xxyy[3]-xxyy[2])*(xxyy[1]-xxyy[0]),
    #        (x1s, x2s, y1s, y2s, yys))
    in_range = tf.logical_or(tf.logical_and(yy1s<=yys, yys<yy2s), tf.logical_and(yy2s<=yys, yys<yy1s))
    realxxs= tf.case((in_range, xxs), tf.zeros_like(xxs))
    return realxxs

    return xxs

if __name__=='__main__':

    print('test glyphDiff...')
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        sq1 = [1/4, 0.5, 1/8, 0.5, 1/2, 0.5, 1, 0.5, -1/2, 0.5]
        sq2 = [1/4, 0.4, 0, 0.5, 1/2, 0.5, 1, 0.5, -1/2, 0.5]
        print(glyphDiffTF(tf.constant(sq1),tf.constant(sq2)).eval())
        #print(glyphDiffTF(tf.constant(Y[0]),tf.constant(Y[1])).eval())



test glyphDiff...
xxs Tensor("add_4:0", shape=(4, 10), dtype=float32)


TypeError: Each entry in pred_fn_pairs must be a 2-tuple